In [1]:
import numpy as np
import torch
from sklearn import ensemble, metrics
import pandas as pd
import csv

In [2]:
def get_header(fn):
    with open(fn, mode='r') as test:
        reader = csv.reader(test)
        rowList = [row[1:] for row in reader]
        header = rowList.pop(0)
        return header
    
def get_full_feat_can(fn):
    header = get_header(fn)
    header.remove('agent')
    header.remove('company')
    return get_listed_feat(fn, get_header(fn))

def get_full_feat_adr(fn):
    header = get_header(fn)
    header.remove('agent')
    header.remove('company')
    header.remove('previous_cancellations')
    header.remove('previous_bookings_not_canceled')
    return get_listed_feat(fn, get_header(fn))
    
    
def get_listed_feat(fn, header):
    vew = []
    a = pd.read_csv(fn)
    for hd in header:
        vew.append(list(a[hd]))
    return np.array(vew).T

def get_label(fn, label): 
    return list(pd.read_csv(fn)[label])

In [3]:
forest_can = ensemble.RandomForestClassifier(n_estimators = 100)
train_can = get_label('data/preprocessed/is_canceled.csv', 'is_canceled')
train_data = get_full_feat_can('data/preprocessed/train_processed.csv')
test_data = get_full_feat_can('data/preprocessed/test_processed.csv')
forest_can.fit(train_data, train_can)
pred_can = forest_can.predict(test_data)
print('done')

done


In [ ]:
forest_adr = ensemble.RandomForestRegressor()
train_adr = get_label('data/preprocessed/adr.csv', 'adr')
train_data = get_full_feat_adr('data/preprocessed/train_processed.csv')
test_data = get_full_feat_adr('data/preprocessed/test_processed.csv')
forest_adr.fit(train_data, train_adr)
pred_adr = forest_adr.predict(test_data)
print('done')

In [15]:
def GetPrediction(data, adr, can):
    assert(len(adr) == len(can))
    assert(len(adr) == len(data))
    date_dict = {}
    a = pd.read_csv('data/preprocessed/test_processed.csv')
    for idx,( _data, _adr, _can)in enumerate(zip(data, adr, can)):
        date = str(a['arrival_date_year'][idx])+'_'
        date += str(a['arrival_date_month'][idx])+'_'
        date += str(a['arrival_date_day_of_month'][idx])
        date_pred = date_dict.get(date)
        date_dict[date] = date_pred if date_pred else 0
        stay_days = a['stays_in_weekend_nights'][idx]
        stay_days += a['stays_in_week_nights'][idx]
        #print(stay_days)
        #print(stay_days)
        date_dict[date] += 0 if _can else _adr*stay_days/10000
        
    print(len(date_dict))
    return date_dict

In [16]:
a = GetPrediction(test_data, pred_adr, pred_can)

153


In [17]:
#print(a)
for key in a:
    print(int(a[key]))

2
2
4
1
3
2
2
4
5
3
3
2
5
4
3
3
4
2
3
2
3
3
3
3
1
1
2
5
5
3
4
4
2
3
4
4
3
4
3
3
4
3
3
4
4
3
3
4
3
3
4
4
2
3
4
3
3
5
4
2
2
4
5
4
4
5
3
2
5
4
4
6
4
3
3
4
4
4
4
5
4
3
3
3
4
4
5
3
3
3
3
6
4
5
5
3
4
4
5
6
6
5
4
4
4
8
6
7
5
3
4
4
6
6
7
5
5
5
5
7
5
7
6
5
7
5
6
6
9
4
5
5
4
7
6
6
5
6
6
6
7
7
6
4
5
5
5
7
5
8
4
3
5
